<a href="https://colab.research.google.com/github/GMorais47/imagem-satelite/blob/main/Projeto_da_Disciplina_Ciclo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar as bibliotecas necessárias no Google Colab
!pip install rasterio gdown numpy matplotlib

# Importar as bibliotecas
import numpy as np
import rasterio
import gdown
import os
import matplotlib.pyplot as plt

# Função para calcular a área baseada em um código específico
def calcular_areas(codigo, data, pixel_area):
    quantidade_pixels = np.sum(data == codigo)
    area_hectares = quantidade_pixels * pixel_area / 10000  # Converter m² para hectares
    return quantidade_pixels, area_hectares

# Função para processar a imagem em partes (blocos) para reduzir uso de memória
def processar_imagem_por_blocos(imagem_path, pixel_area_m2, area_brasil_hectares):
    try:
        with rasterio.open(imagem_path) as src:
            total_pixels = 0
            pixels_sem_dados = 0
            pixels_uteis = 0
            pixels_soja = 0
            pixels_pastagem = 0

            # Leitura por blocos para economizar memória
            for _, window in src.block_windows(1):  # Leitura em blocos
                data = src.read(1, window=window)  # Ler o bloco da imagem

                # Calcular total de pixels
                total_pixels += data.size
                pixels_sem_dados += np.sum(data == 0)  # Contagem de pixels sem dados
                pixels_soja += np.sum(data == 39)  # Contagem de pixels de soja
                pixels_pastagem += np.sum(data == 15)  # Contagem de pixels de pastagem

            # Calcular a quantidade de pixels úteis (com dados)
            pixels_uteis = total_pixels - pixels_sem_dados

            # Calcular o percentual de cobertura útil (desconsiderando pixels sem dados)
            percentual_util = pixels_uteis / total_pixels

            # Calcular a área de soja e pastagem com base no percentual de cobertura
            area_soja_hectares = percentual_util * area_brasil_hectares * (pixels_soja / pixels_uteis)
            area_pastagem_hectares = percentual_util * area_brasil_hectares * (pixels_pastagem / pixels_uteis)

            # *Calcular os percentuais em relação ao Brasil*
            percentual_soja = (area_soja_hectares / area_brasil_hectares) * 100
            percentual_pastagem = (area_pastagem_hectares / area_brasil_hectares) * 100

            # Exibir resultados
            print(f"Total de pixels na imagem: {total_pixels}")
            print(f"Pixels sem dados (código 0): {pixels_sem_dados}")
            print(f"Pixels com plantio de soja (código 39): {pixels_soja}")
            print(f"Área de plantio de soja (proporcional): {area_soja_hectares:.2f} hectares")
            print(f"Pixels com pastagem (código 15): {pixels_pastagem}")
            print(f"Área de pastagem (proporcional): {area_pastagem_hectares:.2f} hectares")
            print(f"Percentual de soja em relação ao território nacional: {percentual_soja:.2f}%")
            print(f"Percentual de pastagem em relação ao território nacional: {percentual_pastagem:.2f}%")
            print(f"Percentual de cobertura útil da imagem: {percentual_util * 100:.2f}%")

            # Exibir a imagem após processamento
            src.close()  # Fechar o arquivo após processamento
            src = rasterio.open(imagem_path)  # Reabrir para exibição
            image_data = src.read(1)  # Ler a imagem novamente

            # Mostrar a imagem usando matplotlib
            plt.figure(figsize=(10, 10))
            plt.imshow(image_data, cmap='gray')
            plt.colorbar()
            plt.title("Imagem de Satélite - Uso e Cobertura do Território Nacional")
            plt.show()

    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")

# Função para baixar o arquivo do Google Drive
def baixar_imagem(link_drive, output_path):
    try:
        file_id = link_drive.split('/d/')[1].split('/view')[0]
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
        print(f"Arquivo baixado com sucesso: {output_path}")
    except Exception as e:
        print(f"Erro ao baixar o arquivo: {e}")

# Parâmetros
link_drive = "https://drive.google.com/file/d/1BrtGByxXu5xqBGOvSBmPWkb_YGZagHav/view?usp=sharing"
output_path = "imagem_satelite.tif"
pixel_area_m2 = 30 * 30  # Exemplo: cada pixel representa 30m x 30m (900 m²)
area_brasil_hectares = 851576700  # Área do Brasil em hectares

# Baixar o arquivo
baixar_imagem(link_drive, output_path)

# Chamar a função principal
if os.path.exists(output_path):
    processar_imagem_por_blocos(output_path, pixel_area_m2, area_brasil_hectares)
else:
    print("Erro: o arquivo não foi baixado corretamente.")